In [4]:
import matplotlib.pyplot as plt

import torch
from torch import nn

In [3]:
DEV = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEV)

cpu


In [ ]:
class AdaptiveVectorNavigation():
    def __init__(self, 
                 forage_time: int, 
                 num_neurons: int,
                 leak_rate  : float) -> None:
        self.feeder_position = torch.randn((2,), device=DEV) # TODO: follow eqs 25, 26, 27
        self.time = 0
        self.forage_time = forage_time
        self.reward_threshold = 10 # TODO: make real value

        self.phi      = 0
        self.speed    = 0
        self.state    = True # outward trip initially
        self.position = torch.zeros((2,), device=DEV) # (x,y)
        self.rewards  = 0
    
        self.neuron_phi            = (2 * torch.pi / num_neurons) * torch.arange(num_neurons, device=DEV)
        self.cosine_kernel         = torch.empty((num_neurons, num_neurons), device=DEV)
        for i in range(num_neurons):
            for j in range(num_neurons):
                self.cosine_kernel[i,j] = torch.cos(self.neuron_phi[i] - self.neuron_phi[j])
        self.previous_memory_layer = torch.zeros((num_neurons,), device=DEV)
        self.leak_rate             = leak_rate
        self.home_vector           = torch.zeros((num_neurons,), device=DEV)
        self.global_vector         = torch.zeros((num_neurons,), device=DEV)

    def update_home_vector(self) -> None:
        head_layer = torch.cos(self.phi - self.neuron_phi)
        gating_layer = nn.ReLU(head_layer - 1 + self.speed)
        memory_layer = nn.ReLU(gating_layer + ((1 - self.leak_rate) * self.previous_memory_layer))
        self.previous_memory_layer = memory_layer
        decoding_layer = nn.ReLU(torch.matmul(self.cosine_kernel, memory_layer)) # actually the head vector
        head_vector_angle = torch.atan(torch.dot(decoding_layer, torch.sin(self.neuron_phi)) / torch.dot(decoding_layer, torch.cos(self.neuron_phi)))
        head_vector_length = torch.sum(decoding_layer) # could be incorrect implementation, hard to understand article
        homing_signal = head_vector_length * torch.sin(head_vector_angle - self.phi - torch.pi)

    def update_global_vector(self) -> None:
        pass

    def update_navigation(self) -> None:
        # update sensory input
        # self.phi, self.speed, self.state, and self.rewards
        self.phi = torch.atan(self.position[1] / self.position[0])
        self.speed = 1 # TODO: make based on change in position over time
        self.state = (self.time < self.forage_time) and (self.rewards < self.reward_threshold)
        self.rewards = nn.ReLU(1 - 5 * torch.norm(self.feeder_position - self.position))

        # update home vector array activities
